# Import modules

In [1]:
import time
import os
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm

In [2]:
# Options for Chrome automated control
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--verbose")
chrome_options.add_experimental_option(
    "prefs",
    {
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing_for_trusted_sources_enabled": False,
        "safebrowsing.enabled": False,
    },
)
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-software-rasterizer")

In [3]:
DRIVER_PATH = 'C:/Users/Yojana.Gadiya/Downloads/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=chrome_options)

# Load file

In [4]:
DATA_DIR = '../data'

In [5]:
surechembl_df = pd.read_parquet(f'{DATA_DIR}/surechembl_dump.pq')
surechembl_df.head(2)

ArrowMemoryError: realloc of size 2147483648 failed

In [6]:
unique_patent_ids = surechembl_df['PATENT_ID'].unique()
len(unique_patent_ids)

1574602

In [8]:
if os.path.exists(f'{DATA_DIR}/patent_code.json'):
    pat_code_mapper = json.load(open(f'{DATA_DIR}/patent_code.json'))
else:
    pat_code_mapper = {}

c = 0

for patent_id in tqdm(unique_patent_ids, total=len(unique_patent_ids)):
    
    if patent_id in pat_code_mapper:
        continue

    # Load patent page
#     print(patent_id)
    driver.get(f'https://www.surechembl.org/document/{patent_id}')
    timeout = 20
    
    try:
        element_present = EC.presence_of_element_located((By.ID, 'classif-ipcr'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        pat_code_mapper[patent_id] = None
        continue
        
    # get IPCR code
    ipcr_data = driver.find_element(By.ID, "classif-ipcr").text.split('\n')
    
    c += 1
    ipcr = set()
    
    for text in ipcr_data:
        if len(text.split()) < 2:  # IPC codes come with alphanumeric characters A61K 18/19
            continue
            
        if not text.split()[0].isalnum():
            continue
        ipcr.add(text)
    
    pat_code_mapper[patent_id] = list(ipcr)
    
    if c == 10:
        with open(f'{DATA_DIR}/patent_code.json', 'w') as f:
            json.dump(pat_code_mapper, f, ensure_ascii=False, indent=2)
        c = 0    


  2%|██▏                                                                                                                                     | 26040/1574602 [00:14<14:23, 1793.45it/s]


KeyboardInterrupt: 

In [ ]:
driver.quit()